In [ ]:
import gmsh
import meshio
import numpy as np
from mpi4py import MPI
from dolfinx import mesh, fem
from dolfinx.io import gmshio
import ufl
from petsc4py import PETSc

gmsh.initialize()
gmsh.model.add("helicopter_blade")

# Ajouter les points de la pale
points = [
    (0.35, 0.15, 0),
    (0, 0.15, 0),
    (0, 0, 0),
    (0.35, 0, 0),
    (6, 0, 0),
    (6, 0.5, 0),
    (0.7, 0.5, 0),
    (0.35, 0.5, 0)
]

for i, (x, y, z) in enumerate(points, start=1):
    gmsh.model.geo.addPoint(x, y, z, 0.1, i)

# Ajouter les lignes droites
gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 5, 4)
gmsh.model.geo.addLine(5, 6, 5)
gmsh.model.geo.addLine(6, 7, 6)

# Ajouter l'arc de cercle
gmsh.model.geo.addCircleArc(7, 8, 1, 9)

# Ajouter la ligne coupant la pale
gmsh.model.geo.addLine(4, 1, 10)

# Définir les boucles de courbes pour les surfaces
surface1_lines = [1, 2, 3, 10]
surface2_lines = [-10, 4, 5, 6, 9]

# Ajouter des surfaces pour marquer les conditions aux limites
gmsh.model.geo.addCurveLoop(surface1_lines, 1)
gmsh.model.geo.addCurveLoop(surface2_lines, 2)

set1 = gmsh.model.geo.addPlaneSurface([1], 1)
set2 = gmsh.model.geo.addPlaneSurface([2], 2)

# Synchroniser la géométrie et générer le maillage
gmsh.model.geo.synchronize()

# Définir la taille caractéristique globale
gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.1)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.1)

gdim = 2
gmsh.model.mesh.generate(gdim)

# Marquer les surfaces
gmsh.model.addPhysicalGroup(2, [1], 1)
gmsh.model.addPhysicalGroup(2, [2], 2)

# Sauvegarder le maillage dans un fichier .msh
gmsh.write("helicopter_blade.msh")

# Initialize MPI
gmsh_model_rank = 0
mesh_comm = MPI.COMM_WORLD

# Convertir le modèle Gmsh en maillage FEniCSx
domain, cell_markers, facet_markers = gmshio.model_to_mesh(gmsh.model, mesh_comm, gmsh_model_rank, gdim=gdim)

# Finaliser Gmsh
gmsh.finalize()

# Lire le fichier .msh avec meshio
msh = meshio.read("helicopter_blade.msh")